In [ ]:
import pandas as pd
from skbio.diversity import beta_diversity
from skbio.stats.distance import permanova
from skbio.stats.distance import permdisp


In [18]:
periods = [('2024-07-11', '2024-07-17'),  # PERIODS FOR JUST CLOVER
           ('2024-07-18', '2024-07-24'), 
           ('2024-07-24', '2024-07-30')
           ]
periods = [(pd.to_datetime(start).date, pd.to_datetime(end).date) for start, end in periods]

def which_period(date):
    for i, (start, end) in enumerate(periods, start=1):
        if start <= date <= end:
            return f"P{i}"
    return np.nan

In [19]:
pi_date_color_counts = pd.read_csv('csvs/pi_date_color_counts.csv', index_col=False)
pi_date_color_counts['date'] = pd.to_datetime(pi_date_color_counts['date']).dt.date
pi_date_color_counts['count_per_10k'] = pi_date_color_counts['count']/10000

threshold = pd.to_datetime('2024-07-30').date()
pi_date_color_counts_clover = pi_date_color_counts[pi_date_color_counts['date'] < threshold]
pi_date_color_counts_clover['period'] = pi_date_color_counts_clover['date'].apply(which_period)
pi_date_color_counts_clover

TypeError: '<=' not supported between instances of 'method' and 'datetime.date'

In [15]:
visit_durations_clover = pd.read_csv('csvs/visit_durations_clover.csv', index_col=False)
visit_durations_clover['date'] = pd.to_datetime(visit_durations_clover['visit_start']).dt.date
visit_durations_clover['period'] = visit_durations_clover['date'].apply(which_period)

visit_durations_clover

NameError: name 'which_period' is not defined